HW 4.2: Recall the Microsoft logfiles data from the async lecture. The logfiles are described are located at:

https://kdd.ics.uci.edu/databases/msweb/msweb.html
http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/

This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.

 Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:

C,"10001",10001   #Visitor id 10001

V,1000,1          #Visit by Visitor 10001 to page id 1000

V,1001,1          #Visit by Visitor 10001 to page id 1001

V,1002,1          #Visit by Visitor 10001 to page id 1002

C,"10002",10002   #Visitor id 10001

V

Note: #denotes comments
to the format:

V,1000,1,C, 10001

V,1001,1,C, 10001

V,1002,1,C, 10001

Write the python code to accomplish this.

In [ ]:
!tail -n 100 anonymous-msweb.data

In [142]:
%%writefile msft_preprocess.py

#!/usr/bin/python
import sys
import re

filename_r = sys.argv[1]
filename_w = sys.argv[2]

print sys.argv
start_C = 0
start_V = 0

with open(filename_r,"r") as fr:
    for line_r in fr:
        line_rs = []
        line_rs = [subs.strip('\"') for subs in re.split(',',line_r.strip())]
        if((start_V==1) and (start_C==1) and line_rs[0]=='V'):
            line_rs.append('C')
            line_rs.append(case_ID)
            with open(filename_w,"a+") as fw:
                fw.write(",".join(line_rs)+"\n")
        elif((start_V==0) and (start_C==1) and line_rs[0] =='V'):
            line_rs.append('C')
            line_rs.append(case_ID)
            with open(filename_w,"a+") as fw:        
                fw.write(",".join(line_rs)+"\n")
                start_V = 1
        elif((start_V==1) and (start_C==1) and line_rs[0] =='C'):
            case_ID = line_rs[1]
            start_V = 0
            start_C = 1
        elif((start_V==0) and (start_C==1) and line_rs[0] =='C'):
            case_ID = line_rs[1]
            start_V = 0
            start_C = 1
        elif(start_V==0 and (start_C==0) and (line_rs[0]=='C')):
            case_ID = line_rs[1]
            start_V = 0
            start_C = 1
        
        

Overwriting msft_preprocess.py


In [143]:
!rm anonymous-msweb.data_mod
!python msft_preprocess.py anonymous-msweb.data anonymous-msweb.data_mod

['msft_preprocess.py', 'anonymous-msweb.data', 'anonymous-msweb.data_mod']


In [ ]:
!cat anonymous-msweb.data_mod 

HW 4.3: Find the 5 most frequently visited pages using MrJob from the output of 4.2 (i.e., transfromed log file).

HW 4.4: Find the most frequent visitor of each page using MrJob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.

HW 4.5 Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

0: Human, where only basic human-human communication is observed.

1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).

2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).

3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )

Check out the preprints of our recent research,
which spawned this dataset:

http://arxiv.org/abs/1505.04342
http://arxiv.org/abs/1508.01843

The main data lie in the accompanying file:

topUsers_Apr-Jul_2014_1000-words.txt

and are of the form:

USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
.
.

where

USERID = unique user identifier
CODE = 0/1/2/3 class code
TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm in MrJob on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

(A) K=4 uniform random centroid-distributions over the 1000 words
(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(D) K=4 "trained" centroids, determined by the sums across the classes.

For (A),  we select 4 users randomly from a uniform distribution [1,...,1,000]
For (B), (C), and (D)  you will have to use data from the auxiliary file: 

topUsers_Apr-Jul_2014_1000-words_summaries.txt

This file contains 5 special word-frequency distributions:

(1) The 1000-user-wide aggregate, which you will perturb for initializations
in parts (B) and (C), and

(2-5) The 4 class-level aggregates for each of the user-type classes (0/1/2/3)

In part (D), just use the (row-normalized) class-level aggregates as 'trained'
starting centroids (the training is already done for you!).

In parts (B) and (C), you will have to perturb the 1000-user aggregate 
(after initially normalizing by its sum, which is also provided).
So if in (B) you want to create 2 perturbations of the aggregate, start
with (1), normalize, and generate 1000 random numbers uniformly 
from the unit interval (0,1) twice (for two centroids), using:

from numpy import random
numbers = random.sample(1000)

Take these 1000 numbers and add them (component-wise) to the 1000-user aggregate,
and then renormalize to obtain one of your aggregate-perturbed initial centroids.
——
For experiments A, B, C and D and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.

Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:

topUsers_Apr-Jul_2014_1000-words_summaries.txt
